In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

Using TensorFlow backend.


In [2]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [3]:
X_train = dataset[0][0][0]
meanX = X_train.mean(axis=1)
stdX = X_train.std(axis = 1)
X_train = X_train.sub(meanX, axis=0)
X_train = X_train.div(stdX, axis = 0)
X_train = X_train.values

y_train = dataset[0][0][1].values

X_test = dataset[1][0][0]
meanX = X_test.mean(axis=1)
stdX = X_test.std(axis = 1)
X_test = X_test.sub(meanX, axis=0)
X_test = X_test.div(stdX, axis = 0)
X_test = X_test.values

y_test = dataset[1][0][1].values

In [4]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
# y_train = get_one_hot(y_train, 2)
# y_test = get_one_hot(y_test, 2)

In [5]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31)
x test shape: (490, 31)
y test shape: (490, 31)


In [6]:
timestep = 10

In [7]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=timestep, sampling_rate=1,
                               batch_size=(X_train.shape[0] - timestep))

In [8]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=timestep, sampling_rate=1,
                               batch_size=(X_test.shape[0] - timestep))

In [9]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [10]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (740, 10, 31)
y train shape: (740, 31)
x test shape: (480, 10, 31)
y test shape: (480, 31)


In [11]:
X_train = X_train.transpose((0,2,1))
X_train = np.reshape(X_train, (X_train.shape[0] * X_train.shape[1], timestep))
y_train = np.reshape(y_train, (y_train.shape[0] * y_train.shape[1], 1))

X_test = X_test.transpose((0,2,1))
X_test = np.reshape(X_test, (X_test.shape[0] * X_test.shape[1], timestep))
y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 1))

In [12]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (22940, 10)
y train shape: (22940, 1)
x test shape: (14880, 10)
y test shape: (14880, 1)


In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train, y_train)

/home/tqa/anaconda3/envs/projet_S5/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tqa/anaconda3/envs/projet_S5/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
from xgboost import XGBClassifier

In [43]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

/home/tqa/anaconda3/envs/projet_S5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tqa/anaconda3/envs/projet_S5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [44]:
predict = clf.predict(X_test)

In [45]:
predict = model.predict(X_test)

In [46]:
b = predict.tolist()

In [47]:
a = y_test.flatten().tolist()

In [48]:
sum(i == j for i,j in zip(a,b))/len(a)

0.5420698924731183

In [49]:
c = np.array(b)

In [50]:
c.shape

(14880,)

In [51]:
d = c.reshape(c.shape[0]//31, 31)

In [52]:
e = y_test.reshape(y_test.shape[0]//31, 31)

In [53]:
d.shape

(480, 31)

In [54]:
e.shape

(480, 31)

In [55]:
d.size

14880

In [56]:
sum(sum(d == e))/d.size

0.5420698924731183